In [10]:
import os
import pandas as pd
import numpy as np
from llm_process_identification_pipeline import create_pipeline 
import time

In [9]:
# Load the Excel file
df = pd.read_excel("dataset.xlsx")

# Create the pipeline instance (this returns the process_tasks function)
pipeline = create_pipeline()

# Helper function to create the expected user_tasks dictionary from a DataFrame row.
def row_to_user_tasks(row):
    # If a value is missing, use an empty string.
    responsible_for = row["Responsible_For"] if pd.notna(row["Responsible_For"]) else ""
    supporting = row["Supporting"] if pd.notna(row["Supporting"]) else ""
    designing = row["Designing"] if pd.notna(row["Designing"]) else ""
    
    # The pipeline expects lists even if the column is just a single string.
    return {
        "responsible_for": [str(responsible_for)],
        "supporting": [str(supporting)],
        "designing": [str(designing)]
    }

# Helper function to generate the LLM input text, similar to what the pipeline uses internally.
def generate_llm_input_text(user_tasks):
    return "\n".join(
        ["Responsible For:"] + user_tasks.get("responsible_for", []) +
        ["Supporting:"] + user_tasks.get("supporting", []) +
        ["Designing:"] + user_tasks.get("designing", [])
    )

In [11]:
# Prepare lists to store new column data
llm_input_list = []
llm_output_list = []

# Iterate through each row in the DataFrame
for idx, row in df.iterrows():
    # Convert row to the user_tasks format
    user_tasks = row_to_user_tasks(row)
    
    # Generate the LLM input text (this is what the pipeline will internally use)
    llm_input_text = generate_llm_input_text(user_tasks)
    
    # Call the RAG pipeline for the current row.
    try:
        output = pipeline(user_tasks)
    except Exception as e:
        output = {"error": str(e)}
    
    llm_input_list.append(llm_input_text)
    llm_output_list.append(output)
    time.sleep(5)


# Add the new columns to the DataFrame
df["LLM Input"] = llm_input_list
df["LLM_Output"] = llm_output_list

# Save the updated DataFrame to a new Excel file
df.to_excel("dataset_with_llm_results.xlsx", index=False)
print("Processing complete. Results saved to dataset_with_llm_results.xlsx.")

Language detection result: is_german=False is_english=True detected_language='English'
Validation result: is_valid_responsible_for=True is_valid_supporting=True is_valid_designing=True message='All inputs are valid and meaningful tasks for a Systems Engineer. The responsibilities include formal compliance verification, driving certification programs, and participating in rulemaking activities. Supporting tasks involve assisting engineering teams and guiding compliance with standards. Designing tasks focus on creating certification strategies and test planning.'
Token count for process identification: 1752
Identified Processes: ['Validation process', 'Verification process', 'Risk management process', 'Stakeholder needs and requirements definition process', 'System requirements definition process', 'Design definition process', 'Implementation process', 'Integration process', 'Transition process']
Retrieval Query: Risk management process The purpose of the risk management process is to id

/Users/derikroby/Desktop/Thesis/Implementation/surveyapp/Validation/llm_evaluation_results/llm_process_identification_pipeline.py:415: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(retrieval_query)


Retrieved docs: [Document(metadata={}, page_content='Process: Risk management process\nProcess Description: The purpose of the risk management process is to identify, analyse, treat, and monitor the riskscontinually.The risk management process systematically addresses uncertainty throughout the life cycle of asystem product or service towards achieving objectives.\n  Activity: Plan risk management\n    Task: Define the risk management strategy\n    Task: Define and record the context of the risk management process.\n  Activity: Maintain the risk profile\n    Task: Define and record the risk thresholds and conditions.\n    Task: Establish and maintain a risk profile\n    Task: Periodically provide the relevant risk profile to stakeholders\n  Activity: Analyse risks\n    Task: Identify risks in the categories described in the risk management context.\n    Task: Estimate the likelihood of occurrence and consequences of each identified risk\n    Task: Evaluate each risk against its risk th

In [12]:
df

,Dataset_Number,Responsible_For,Supporting,Designing,Gold_Responsible_For,Gold_Supporting,Gold_Designing,LLM_Output_Responsible_For,LLM_Output_Supporting,LLM_Output_Designing,LLM Input,LLM_Output
0,1,1. Formal compliance verification for the Volo...,1. Assisting the Avionics Engineering Team in ...,1. Certification strategy design: Defining how...,"verification process, Project planning process...","risk management process, life cycle model mana...","System architecture definition process, Stakeh...",NaN,NaN,NaN,Responsible For:\n1. Formal compliance verific...,"{'status': 'success', 'result': processes=[ISO..."
1,2,1. Führung und Weiterentwicklung des siebenköp...,1. Zusammenarbeit mit anderen Abteilungen (z. ...,1. Design von Testprozessen: Entwicklung von s...,"Human resource management process, Verificatio...","Life cycle model management process, Human res...",Verification process; Validation process; Qual...,NaN,NaN,NaN,Responsible For:\n1. Führung und Weiterentwick...,"{'status': 'success', 'result': processes=[ISO..."
2,3,1. Performing testing and validation of ADAS h...,1. Benchmarking activities: Assisting in compe...,1. Defining use cases and test scenarios in co...,"Validation process,\nQuality assurance process...","Knowledge management process, Information mana...",Verification process; Validation process Verif...,NaN,NaN,NaN,Responsible For:\n1. Performing testing and va...,"{'status': 'success', 'result': processes=[ISO..."
3,4,\n1. Erstellung von Testplänen und Testprozedu...,1. Zusammenarbeit mit internen Teams (z. B. En...,1. Design von Testprozessen: Entwicklung stand...,"1. Verification, Validation\n2 Verification, V...","1. no mapping\n2 Verification, Validation (may...","1 Verification, Validation (maybe Integration)...",NaN,NaN,NaN,Responsible For:\n\n1. Erstellung von Testplän...,"{'status': 'success', 'result': processes=[ISO..."
4,5,1. Fachliche und disziplinarische Führung des ...,1. Zusammenarbeit mit Entwicklungsabteilungen ...,1. Entwicklung von Methoden zur Produktqualitä...,Information management process; Project planni...,nformation management process\nRisk management...,Quality assurance process\nVerification & Vali...,NaN,NaN,NaN,Responsible For:\n1. Fachliche und disziplinar...,"{'status': 'success', 'result': processes=[ISO..."
5,6,1. Preparation of technical repercussion sheet...,1. Interface management with stakeholders (e.g...,1. Implementation of change management process...,Life cycle model management process; Configura...,Stakeholder needs and requirements definition ...,Implementation process Configuration managemen...,NaN,NaN,NaN,Responsible For:\n1. Preparation of technical ...,"{'status': 'success', 'result': processes=[ISO..."
6,7,1. Technical and timely implementation of cont...,"1. Acting as the interface between clients, su...","1. Optimizing site management workflows (e.g.,...",. Integration process\n1. project planning pro...,. Traqnsition process\n1. Project assessment a...,1. Transition process\n1. operation process\n1...,NaN,NaN,NaN,Responsible For:\n1. Technical and timely impl...,"{'status': 'success', 'result': processes=[ISO..."
7,8,1. Lead programs to secure and optimize IoT-en...,1. Act as a bridge between technical teams (e....,1. Design resilience frameworks for renewable ...,1. design definition process\ndesgin definitio...,hart to map\n1. Risk management process 1. ha...,1. systems analysis process 1. systesm design ...,NaN,NaN,NaN,Responsible For:\n1. Lead programs to secure a...,"{'status': 'success', 'result': processes=[ISO..."
8,9,"1. Requirement Gathering & Analysis: Elicit, d...",1. Stakeholder Coordination: Act as a liaison ...,1. Designing System Requirements: Develop stru...,1. system definition requirement process 1. 2....,. SNRD\n1. project planning process\n1. knowle...,1. system architecture definition process\n1. ...,NaN,NaN,NaN,Responsible For:\n1. Requirement Gathering & A...,"{'status': 'success', 'result': processes=[ISO..."
9,10,1. Develop t